In [1]:
from data import CocoDatasetManager

In [4]:
# Example usage
coco = CocoDatasetManager()

# Later, load the saved datasets
loaded_train_dataset = coco.load_saved_dataset("/workspaces/anaconda/src/data/train")
loaded_val_dataset = coco.load_saved_dataset("/workspaces/anaconda/src/data/test")


Importing samples...
 100% |███████████████████| 50/50 [7.0ms elapsed, 0s remaining, 7.1K samples/s]       
Import complete
Importing samples...
 100% |███████████████████| 50/50 [6.9ms elapsed, 0s remaining, 7.2K samples/s]       
Import complete


In [5]:
loaded_train_dataset

Name:        2023.11.19.15.49.17
Media type:  image
Num samples: 50
Persistent:  False
Tags:        []
Sample fields:
    id:            fiftyone.core.fields.ObjectIdField
    filepath:      fiftyone.core.fields.StringField
    tags:          fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)